## Clean up
Please make sure to comment the below section if you are planning to use the Knowledge Base that you created above for building your RAG application.
If you only wanted to try out creating the KB using SDK, then please make sure to delete all the resources that were created as you will be incurred cost for storing documents in OSS index.

#### Delete KnowledgeBase and delete resources after completing all the notebooks.


In [1]:
%store -r

In [2]:
import boto3
import time

boto3_session = boto3.Session()
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=boto3_session.region_name)
aoss_client = boto3.client('opensearchserverless')
s3_client = boto3_session.client('s3', region_name=boto3_session.region_name)
iam_client = boto3.client("iam")

#### Delete Bedrock KnowledgeBase data sources

In [3]:
# fetch data source(s)associated with kb
response = bedrock_agent_client.list_data_sources(knowledgeBaseId=kb_id)
data_sources_list = [ds_summary for ds_summary in response['dataSourceSummaries']]

for idx, ds in enumerate(data_sources_list):
    bedrock_agent_client.delete_data_source(dataSourceId = ds["dataSourceId"], knowledgeBaseId=ds["knowledgeBaseId"])
    time.sleep(10)


#### Remove KnowledgeBases and OpenSearch Collection

In [4]:
# Fetch kb execution role
response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId=kb_id)
kb_role_name = response['knowledgeBase']['roleArn'].split("/")[-1]

In [5]:
# fetch all attched policies with kb execution role
kb_attached_role_policies_response = iam_client.list_attached_role_policies(
    RoleName=kb_role_name)
kb_attached_role_policies = kb_attached_role_policies_response['AttachedPolicies']

In [ ]:
bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=kb_id)
aoss_client.delete_collection(id=collection['createCollectionDetail']['id'])
aoss_client.delete_access_policy(type="data", name=access_policy['accessPolicyDetail']['name'])
aoss_client.delete_security_policy(type="network", name=network_policy['securityPolicyDetail']['name'])
aoss_client.delete_security_policy(type="encryption", name=encryption_policy['securityPolicyDetail']['name'])

#### Delete role and policies


In [7]:
for policy in kb_attached_role_policies:
    iam_client.detach_role_policy(
            RoleName=kb_role_name,
            PolicyArn=policy['PolicyArn']
    )

In [ ]:
iam_client.delete_role(RoleName=kb_role_name)

In [9]:
for policy in kb_attached_role_policies:
    iam_client.delete_policy(PolicyArn=policy['PolicyArn'])

#### Delete S3 objects

In [ ]:
objects = s3_client.list_objects(Bucket=bucket_name)
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
s3_client.delete_bucket(Bucket=bucket_name)